In [ ]:
import nltk 
from nltk.stem.lancaster import LancasterStemmer
import h5py
import numpy
import tensorflow as tf
import random
import json
import spacy
from nltk.stem.porter import *

class Chatbot:

    showWorking = False
    train_again=False
    vocabulary=[]
    labels=[]
    responseDictionary = {}
    stemmer = PorterStemmer()
    
    def __init__(self,train_again=False):
        Chatbot.train_again = train_again
        
    spacy_nlp = spacy.load('en_core_web_sm')
    spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
    customize_stop_words = ['.', '?','!']
    for w in customize_stop_words:
        spacy_nlp.vocab[w].is_stop = True

    def get_model(self):

        if not Chatbot.train_again:
            with open ('LabelList.json') as fl:
                Chatbot.labels = json.load(fl)
            
            with open ('VocabularyList.json') as fv:
                Chatbot.vocabulary = json.load(fv)
                
            with open ('responseDictionary.json') as fr:
                Chatbot.responseDictionary = json.load(fr)
                
            model= tf.keras.models.load_model("chatBotold.h5")
        else:
            with open('CommonIntentions.json') as file:
                data = json.load(file)

            docs_x = []
            docs_y = []

            for intent in data['intents']:
                
                Chatbot.responseDictionary[intent['tag']] = []
                responseList=[]
                for response in intent['responses']:
                    responseList.append(response)
                    
                Chatbot.responseDictionary[intent['tag']] = responseList
                
                for pattern in intent['patterns']:
                    wrds = remove_stop_words_get_lemmas(pattern)
                    if len(wrds)>0:
                        Chatbot.vocabulary.extend(wrds)
                        docs_x.append(wrds)
                        docs_y.append(intent['tag'])               

                if intent['tag'] not in Chatbot.labels:
                    Chatbot.labels.append(intent['tag'])
                    
            with open('responseDictionary.json', 'w') as fp:
                json.dump(Chatbot.responseDictionary, fp)
                
            Chatbot.vocabulary = sorted(list(set(Chatbot.vocabulary)))
            Chatbot.labels = sorted(Chatbot.labels)
            training = []
            output = []
            out_empty = [0 for _ in range(len(Chatbot.labels))]

            for x, doc in enumerate(docs_x):
                bag = []

                for w in Chatbot.vocabulary:
                    if w in doc:
                        bag.append(1)
                    else:
                        bag.append(0)

                output_row = out_empty[:]
                output_row[Chatbot.labels.index(docs_y[x])] = 1

                training.append(bag)
                output.append(output_row)

            training = numpy.array(training)
            output = numpy.array(output)


            model = tf.keras.Sequential()
            for layer in range(3):
            # Adds a densely-connected layer with specified units to the model:
                model.add(tf.keras.layers.Dense(10, activation='relu'))

        # Add a sigmoid layer with 1 output units:
            model.add(tf.keras.layers.Dense(len(output[0]), activation='softmax'))
            model.compile(optimizer='adam',loss='categorical_crossentropy',metric=['accuracy'])
            model.fit(training, output, epochs=300)
            model.save("chatBotold.h5")
            
            with open('LabelList.json', 'w') as fl:
                json.dump(Chatbot.labels, fl)
            with open('VocabularyList.json', 'w') as fv:
                json.dump(Chatbot.vocabulary, fv)
        return model

    def get_yes_list(self):
        return ['yes','sure','ok','okay','for sure','sure','totally','yep','ya','yeah','yup','certainly',
                         'definitely','of course','gladly','absolutely','indeed']
    
    def chat(self):
        
        print("Do you want to see internal processing steps:")
        choice = input("Enter you choice : ")
        if choice.lower() in self.get_yes_list():
            Chatbot.showWorking=True
        model = self.get_model()
        print("Start talking with the bot (type quit to stop)!")
        while True:
            inp = input("You: ")
            if inp.lower() == "quit":
                break
            results = model.predict(bag_of_words(inp))
            results_index = numpy.argmax(results)
            tag = Chatbot.labels[results_index]
            if Chatbot.showWorking:
                print("Highest intent proability:",results[0][results_index])
                print("Intent of sentence : ",tag)
            responses = Chatbot.responseDictionary[tag]
            print("chatbot:",random.choice(responses))
            
def remove_stop_words_get_lemmas(sentence):
        #doc = Chatbot.spacy_nlp(sentence)
        #lemmaList = [token.lemma_ for token in doc if not token.is_stop]
        s_words = nltk.word_tokenize(sentence)
        s_words = [Chatbot.stemmer.stem(word.lower()) for word in s_words]
        if Chatbot.showWorking:
            #print('Lemma :',lemmaList)
            print('Stems :',s_words)
        return s_words

def bag_of_words(sentence):
    bags = []
    #creating hot vector
    bag = [0 for _ in range(len(Chatbot.vocabulary))] 
    s_words = remove_stop_words_get_lemmas(sentence)

    for se in s_words:
        for i, w in enumerate(Chatbot.vocabulary):
            if w == se:
                bag[i] = 1
                
    if Chatbot.showWorking:
        print('Vector Representation of input sentence is:',bag)
            
    bags.append(bag)
    bags = numpy.array(bags)
    return bags
        
def main():
    chatbot = Chatbot(True)
    chatbot.chat()

if __name__ == '__main__':
    main()
    


Do you want to see internal processing steps:
Enter you choice : yes
Stems : ['tell', 'me', 'some', 'car', 'which', 'are', 'in', 'my', 'budget']
Stems : ['cheap', 'car']
Stems : ['cheapest', 'car']
Stems : ['afford']
Stems : ['low', 'budget']
Stems : ['lower', 'end', 'car']
Stems : ['low', 'maintainc', 'car']
Stems : ['low', 'cost', 'car']
Stems : ['afford', 'luxuri', 'car']
Stems : ['high', 'class', 'car']
Stems : ['high', 'end', 'car']
Stems : ['expens', 'car']
Stems : ['what', 'car', 'i', 'should', 'buy']
Stems : ['which', 'car', 'to', 'buy']
Stems : ['trend', 'car']
Stems : ['differ', 'compani', 'of', 'car']
Stems : ['which', 'is', 'the', 'latest', 'car']
Stems : ['tell', 'me', 'some', 'mileag', 'car']
Stems : ['what', 'car', 'i', 'should', 'buy']
Stems : ['tell', 'me', 'some', 'good', 'car', 'name']
Stems : ['tell', 'me', 'some', 'car', 'compani', 'name']
Stems : ['what', 'is', 'a', 'good', 'car', 'for', 'student']
Stems : ['good', 'car', 'for', 'elder']
Stems : ['good', 'car', 'f

Stems : ['whi', 'to', 'use', 'you']
Stems : ['what', 'i', 'will', 'get', 'out', 'of']
Stems : ['pro', 'of', 'chatbot']
Stems : ['tell', 'me', 'someth', 'bad', 'about', 'you']
Stems : ['case', 'where', 'chatbot', 'fail']
Stems : ['chatbot']
Stems : ['see', 'you', 'later']
Stems : ['bye']
Stems : ['see', 'you', 'soon']
Stems : ['talk', 'to', 'you', 'later']
Stems : ['catch', 'up', 'later']
Stems : ['will', 'meet', 'again']
Stems : ['good', 'bye']
Stems : ['will', 'talk', 'later']
Stems : ['nice', 'meet', 'talk', 'with', 'you']
Stems : ['will', 'catch', 'you', 'later']
Stems : ['quit']
Stems : ['byye']
Stems : ['catch', 'you', 'later']
Stems : ['what', "'s", 'the', 'weather', 'today', '?']
Stems : ['are', 'you', 'awar', 'of', 'the', 'weather', 'today', '?']
Stems : ['are', 'you', 'awar', 'of', 'weather', 'now', '?']
Stems : ['are', 'you', 'awar', 'of', 'the', 'weather', 'today', '?']
Stems : ['what', "'s", 'the', 'weather', 'now', '?']
Stems : ['what', 'is', 'the', 'weather', 'predict', '

Epoch 36/300
308/308 [==============================] - ETA: 0s - loss: 1.997 - 0s 81us/sample - loss: 2.2646
Epoch 37/300
308/308 [==============================] - ETA: 0s - loss: 2.534 - 0s 110us/sample - loss: 2.2107
Epoch 38/300
308/308 [==============================] - ETA: 0s - loss: 2.360 - 0s 143us/sample - loss: 2.1565
Epoch 39/300
308/308 [==============================] - ETA: 0s - loss: 2.338 - 0s 101us/sample - loss: 2.0993
Epoch 40/300
308/308 [==============================] - ETA: 0s - loss: 1.783 - 0s 101us/sample - loss: 2.0455
Epoch 41/300
308/308 [==============================] - ETA: 0s - loss: 2.094 - 0s 114us/sample - loss: 1.9914
Epoch 42/300
308/308 [==============================] - ETA: 0s - loss: 2.014 - 0s 91us/sample - loss: 1.9380
Epoch 43/300
308/308 [==============================] - ETA: 0s - loss: 1.845 - 0s 110us/sample - loss: 1.8858
Epoch 44/300
308/308 [==============================] - ETA: 0s - loss: 1.869 - 0s 81us/sample - loss: 1.8359
Epoc

308/308 [==============================] - ETA: 0s - loss: 0.352 - 0s 101us/sample - loss: 0.2421
Epoch 110/300
308/308 [==============================] - ETA: 0s - loss: 0.103 - 0s 94us/sample - loss: 0.2362
Epoch 111/300
308/308 [==============================] - ETA: 0s - loss: 0.338 - 0s 127us/sample - loss: 0.2316
Epoch 112/300
308/308 [==============================] - ETA: 0s - loss: 0.151 - 0s 130us/sample - loss: 0.2235
Epoch 113/300
308/308 [==============================] - ETA: 0s - loss: 0.198 - 0s 101us/sample - loss: 0.2189
Epoch 114/300
308/308 [==============================] - ETA: 0s - loss: 0.220 - 0s 153us/sample - loss: 0.2129
Epoch 115/300
308/308 [==============================] - ETA: 0s - loss: 0.186 - 0s 117us/sample - loss: 0.2079
Epoch 116/300
308/308 [==============================] - ETA: 0s - loss: 0.263 - 0s 107us/sample - loss: 0.2029
Epoch 117/300
308/308 [==============================] - ETA: 0s - loss: 0.181 - 0s 123us/sample - loss: 0.1948
Epoch 1

308/308 [==============================] - ETA: 0s - loss: 0.089 - 0s 94us/sample - loss: 0.0596
Epoch 183/300
308/308 [==============================] - ETA: 0s - loss: 0.083 - 0s 101us/sample - loss: 0.0596
Epoch 184/300
308/308 [==============================] - ETA: 0s - loss: 0.080 - 0s 88us/sample - loss: 0.0593
Epoch 185/300
308/308 [==============================] - ETA: 0s - loss: 0.089 - 0s 101us/sample - loss: 0.0613
Epoch 186/300
308/308 [==============================] - ETA: 0s - loss: 0.038 - 0s 140us/sample - loss: 0.0570
Epoch 187/300
308/308 [==============================] - ETA: 0s - loss: 0.052 - 0s 120us/sample - loss: 0.0582
Epoch 188/300
308/308 [==============================] - ETA: 0s - loss: 0.051 - 0s 104us/sample - loss: 0.0547
Epoch 189/300
308/308 [==============================] - ETA: 0s - loss: 0.023 - 0s 101us/sample - loss: 0.0565
Epoch 190/300
308/308 [==============================] - ETA: 0s - loss: 0.080 - 0s 107us/sample - loss: 0.0546
Epoch 19

308/308 [==============================] - ETA: 0s - loss: 0.026 - 0s 97us/sample - loss: 0.0379
Epoch 256/300
308/308 [==============================] - ETA: 0s - loss: 0.016 - 0s 71us/sample - loss: 0.0376
Epoch 257/300
308/308 [==============================] - ETA: 0s - loss: 0.051 - 0s 78us/sample - loss: 0.0371
Epoch 258/300
308/308 [==============================] - ETA: 0s - loss: 0.011 - 0s 104us/sample - loss: 0.0360
Epoch 259/300
308/308 [==============================] - ETA: 0s - loss: 0.013 - 0s 120us/sample - loss: 0.0368
Epoch 260/300
308/308 [==============================] - ETA: 0s - loss: 0.024 - 0s 91us/sample - loss: 0.0371
Epoch 261/300
308/308 [==============================] - ETA: 0s - loss: 0.010 - 0s 104us/sample - loss: 0.0359
Epoch 262/300
308/308 [==============================] - ETA: 0s - loss: 0.009 - 0s 94us/sample - loss: 0.0357
Epoch 263/300
308/308 [==============================] - ETA: 0s - loss: 0.026 - 0s 101us/sample - loss: 0.0363
Epoch 264/3

Vector Representation of input sentence is: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Highest intent proability: 0.59356517
Intent o

Vector Representation of input sentence is: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Highest intent proability: 0.9964796
Intent of

Vector Representation of input sentence is: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Highest intent proability: 0.7566119
Intent of

Vector Representation of input sentence is: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Highest intent proability: 0.9770511
Intent of

In [ ]:
import spacy
from importlib import reload 
reload(spacy)
nlp = spacy.load('en_core_web_sm')

doc = nlp(u"Apples and oranges are similar. Boots and hippos aren't.")
for token in doc:
    print(token, token.lemma_)

In [ ]:
import spacy

while (True):

    doc = input("Enter :")

    if doc=='quit':
        break
    spacy_nlp = spacy.load('en_core_web_sm')
    spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
    
    doc = spacy_nlp(doc)
    customize_stop_words = [
        '.', '?','!'
    ]
    
    for w in customize_stop_words:
        spacy_nlp.vocab[w].is_stop = True

    tokens = [token.lemma_ for token in doc if not token.is_stop]

    print(tokens)




In [ ]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')
print(spacy.lang.en.stop_words.STOP_WORDS)

In [ ]:
def bag_of_words(s, words):
    bags = []
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    #print('tokenizer:',s_words)
    
    print('lemmarizer:',get_lemmas(s))
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    bags.append(bag)
    bags = numpy.array(bags)
    #return numpy.array(bag)
    return bags

In [ ]:
import pickle

    
with open ('responseDictionary', 'rb') as fp:
    itemlist2 = pickle.load(fp)
    
print(itemlist2.keys())

In [ ]:
import nltk 
#from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import *
stemmer = PorterStemmer()
sentence = 'what are you'
s_words = nltk.word_tokenize(sentence)
s_words = [stemmer.stem(word.lower()) for word in s_words]

print(s_words)


    


In [ ]:
dict = {'asds': 'Geeks', 'b': 'For', 'c': 'geeks'} 

def getList(dict): 
      
    return [*dict] 

print(dict['dhjjh']) 